In [10]:
# Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = 'day{0:02d}.txt'.format(day)
    return open(filename)

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

In [2]:
assert tuple(transpose(((1, 2, 3), (4, 5, 6)))) == ((1, 4), (2, 5), (3, 6))
assert first('abc') == first(['a', 'b', 'c']) == 'a'
assert cat(['a', 'b', 'c']) == 'abc'
assert (groupby(['test', 'one', 'two', 'three', 'four'], key=len) 
        == {3: ['one', 'two'], 4: ['test', 'four'], 5: ['three']})

In [3]:
## Advent Day 01

Point = complex             
N, S, E, W = 1j, -1j, 1, -1 # Unit vectors for headings

def distance(point): 
    "City block distance between point and the origin."
    return abs(point.real) + abs(point.imag)

def how_far(moves):
    "After following moves, how far away from the origin do we end up?"
    loc, heading = 0, N # Begin at origin, heading North
    for (turn, dist) in parse(moves):
        heading *= turn
        loc += heading * dist
    return distance(loc)

def parse(text):
    "Return a list of (turn, distance) pairs from text of form 'R2, L42, ...'"
    turns = dict(L=N, R=S)
    return [(turns[RL], int(d))
           for (RL, d) in re.findall(r'(R|L)(\d+)', text)]

assert distance(Point(3, 4)) == 7 # City block distance; Euclidean distance would be 5
assert parse('R2, L42') == [(S, 2), (N, 42)]
assert how_far("R2, L3") == 5
assert how_far("R2, R2, R2") == 2
assert how_far("R5, L5, R5, R3") == 12

print(how_far(Input(1).read()))

def visited_twice(text):
    "Following moves in text, find the first location we visit twice, and return the distance to it."
    loc, heading = 0, N # Begin at origin, heading North
    visited = {loc}
    for (turn, dist) in parse(text):
        heading *= turn
        for i in range(dist):
            loc += heading
            if loc in visited:
                return distance(loc)
            visited.add(loc)

assert visited_twice("R8, R4, R4, R8") == 4
assert visited_twice("R8, R4, R4, L8") == None
assert visited_twice("R8, R0, R1") == 7

print(visited_twice(Input(1).read()))

243.0
142.0


In [5]:
## Advent Day 02

Keypad = str.split

keypad = Keypad("""
.....
.123.
.456.
.789.
.....
""")

assert keypad[2][2] == '5'

off = '.'

def decode(instructions, x=2, y=2):
    """Follow instructions, keeping track of x, y position, and
    yielding the key at the end of each line of instructions."""
    for line in instructions:
        for C in line:
            x, y = move(C, x, y)
        yield keypad[y][x]

def move(C, x, y):
    "Make the move corresponding to this character (L/R/U/D)"
    if   C == 'L' and keypad[y][x-1] is not off: x -= 1
    elif C == 'R' and keypad[y][x+1] is not off: x += 1
    elif C == 'U' and keypad[y-1][x] is not off: y -= 1
    elif C == 'D' and keypad[y+1][x] is not off: y += 1
    return x, y

assert move('U', 2, 2) == (2, 1)
assert move('U', 2, 1) == (2, 1)
assert cat(decode("ULL RRDDD LURDL UUUUD".split())) == '1985'

print(cat(decode(Input(2))))

keypad = Keypad("""
.......
...1...
..234..
.56789.
..ABC..
...D...
.......
""")

assert keypad[3][1] == '5'

print(cat(decode(Input(2), x=1, y=3)))

99332
DD483


In [6]:
## Advent Day 03

def is_triangle(sides):
    "Do these side lengths form a valid triangle?"
    x, y, z = sorted(sides)
    return z < x + y

def parse_ints(text): 
    "All the integers anywhere in text."
    return [int(x) for x in re.findall(r'\d+', text)]

triangles = [parse_ints(line) for line in Input(3)]

print(sum(map(is_triangle, triangles)))

def invert(triangles):
    "Take each 3 lines and transpose them."
    for i in range(0, len(triangles), 3):
        yield from transpose(triangles[i:i+3])

print(sum(map(is_triangle, invert(triangles))))

862
1577


In [8]:
## Advent Day 04

def parse(line): 
    "Return (name, sector, checksum)."
    return re.match(r"(.+)-(\d+)\[([a-z]+)\]", line).groups()

def sector(line):
    "Return the sector number if valid, or 0 if not."
    name, sector, checksum = parse(line)
    return int(sector) if valid(name, checksum) else 0

def valid(name, checksum):
    "Determine if name is valid according to checksum."
    counts = Counter(name.replace('-', '')) 
    # Note: counts.most_common(5) doesn't work because it breaks ties arbitrarily.
    letters = sorted(counts, key=lambda L: (-counts[L], L))
    return checksum == cat(letters[:5])

assert  parse('aaaaa-bbb-z-y-x-123[abxyz]') == ('aaaaa-bbb-z-y-x', '123', 'abxyz')
assert sector('aaaaa-bbb-z-y-x-123[abxyz]') == 123
assert  valid('aaaaa-bbb-z-y-x', 'abxyz')

print(sum(map(sector, Input(4))))

def decrypt(line):
    "Decrypt the line (shift the name by sector; discard checksum)."
    name, sector, _ = parse(line)
    return shift(name, int(sector)) + ' ' + sector

def shift(text, N, alphabet='abcdefghijklmnopqrstuvwxyz'):
    "Shift cipher: letters in text rotate forward in alphabet by N places."
    N = N % len(alphabet)
    tr = str.maketrans(alphabet, alphabet[N:] + alphabet[:N])
    return text.translate(tr)

assert shift('hal', 1) == 'ibm'
assert shift('qzmt-zixmtkozy-ivhz', 343) == 'very-encrypted-name'

print(grep("north", map(decrypt, Input(4))))

245102
northpole-object-storage 324
None


In [11]:
## Advent Day 05

import hashlib

door = "wtnhxymk"

def find_password(door):
    "First 8 sixth digits of md5 hashes of door+i that begin with '00000'."
    password = ''
    for i in range(BIG):
        x = hashlib.md5(bytes(door + str(i), 'utf-8')).hexdigest()
        if x.startswith('00000'):
            password += x[5]
            print(i, x, password) # Just to see something happen
            if len(password) == 8: 
                return password

find_password(door)

def find_tougher_password(door):
    "For md5 hashes that begin with '00000', the seventh digit goes in the sixth-digit slot of the password."
    password = [off] * 8
    for i in range(BIG):
        x = hashlib.md5(bytes(door + str(i), 'utf-8')).hexdigest()
        if x.startswith('00000'):
            index = int(x[5], 16)
            if index < 8 and password[index] is off:
                password[index] = x[6]
                print(i, x, cat(password)) # Just to see something happen
            if off not in password:
                return cat(password)

%time find_tougher_password(door)

2231254 0000027b9705c7e6fa3d4816c490bbfd 2
2440385 00000468c8625d85571d250737c47b5a 24
2640705 0000013e3293b49e4c78a5b43b21023b 241
3115031 0000040bbe4509b48041007dec6123bd 2414
5045682 00000b11810477f9e49840991fb2151e 2414b
8562236 00000cc461c8945671046cf632be4473 2414bc
9103137 000007c1da6865df78b2c0addf28913d 2414bc7
9433034 00000700ce8beb0a8ffc83fa9986d577 2414bc77
2231254 0000027b9705c7e6fa3d4816c490bbfd ..7.....
2440385 00000468c8625d85571d250737c47b5a ..7.6...
2640705 0000013e3293b49e4c78a5b43b21023b .37.6...
9103137 000007c1da6865df78b2c0addf28913d .37.6..c
13753308 0000050301b17d598b52e2a343b80c95 .37.60.c
13976178 000006fe7545b487de2d003f3d4e1114 .37.60fc
19808390 000003e432ea631581aefcce573d56dd .37e60fc
27712456 00000048d155e2c930602533209b0154 437e60fc
Wall time: 39.1 s


'437e60fc'

In [13]:
## Advent Day 06

counts = [Counter(col) for col in transpose(Input(6).read().split())]
print(cat(c.most_common(1)[0][0] for c in counts))
print(cat(c.most_common()[-1][0] for c in counts))

gebzfnbt
fykjtwyn


In [15]:
## Advent Day 07

def abba(text):           return any(a == d != b == c for (a, b, c, d) in subsequences(text, 4))
def subsequences(seq, n): return [seq[i:i+n] for i in range(len(seq) + 1 - n)]
def segment(line):        return re.split(r'\[|\]', line)
def outsides(segments):   return ', '.join(segments[0::2])
def insides(segments):    return ', '.join(segments[1::2])
def tls(segments):        return abba(outsides(segments)) and not abba(insides(segments))

print(sum(tls(segment(line)) for line in Input(7)))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def ssl(segments): 
    "Is there an ABA outside brackets, and the corresponding BAB inside?"
    outs, ins = outsides(segments), insides(segments)
    return any(a+b+a in outs and b+a+b in ins
               for a in alphabet for b in alphabet if a != b)

print(sum(ssl(segment(line)) for line in Input(7)))

115
231


In [16]:
## Advent Day 08

def interpret(cmd, screen):
    "Interpret this command to mutate screen."
    A, B = map(int, re.findall(r'(\d+)', cmd)) # There should be 2 numbers on every command line
    if cmd.startswith('rect'):
        screen[:B, :A] = 1
    elif cmd.startswith('rotate row'):
        screen[A, :] = rotate(screen[A, :], B)
    elif cmd.startswith('rotate col'):
        screen[:, A] = rotate(screen[:, A], B)

def rotate(items, n): return np.append(items[-n:], items[:-n])

def Screen(): return np.zeros((6, 50), dtype=np.int)

def run(commands, screen):
    "Do all the commands and return the final pixel array."
    for cmd in Input(8):
        interpret(cmd, screen)   
    return screen

screen = run(Input(8), Screen())
print(np.sum(screen))

for row in screen:
    print(cat(' @'[pixel] for pixel in row))

121
@@@  @  @ @@@  @  @  @@  @@@@  @@  @@@@  @@@ @    
@  @ @  @ @  @ @  @ @  @ @    @  @ @      @  @    
@  @ @  @ @  @ @  @ @    @@@  @  @ @@@    @  @    
@@@  @  @ @@@  @  @ @    @    @  @ @      @  @    
@ @  @  @ @ @  @  @ @  @ @    @  @ @      @  @    
@  @  @@  @  @  @@   @@  @@@@  @@  @@@@  @@@ @@@@ 


In [18]:
## Advent Day 09

matcher = re.compile(r'[(](\d+)x(\d+)[)]').match # e.g. matches "(2x5)" as ('2', '5')

def decompress(s):
    "Decompress string s by interpreting '(2x5)' as making 5 copies of the next 2 characters."
    s = re.sub(r'\s', '', s) # "whitespace is ignored"
    result = []
    i = 0
    while i < len(s):
        m = matcher(s, i)
        if m:
            i = m.end()                # Advance to end of '(CxR)' match
            C, R = map(int, m.groups())
            result.append(s[i:i+C] * R) # Collect the C characters, repeated R times
            i += C                      # Advance past the C characters           
        else:
            result.append(s[i])         # Collect 1 regular character
            i += 1                      # Advance past it
    return cat(result)

print(len(decompress(Input(9).read())))

def decompress_length(s):
    """Decompress string s by interpreting '(2x5)' as making 5 copies of the next 2 characters.
    Recursively decompress these next 5 characters. Return the length of the decompressed string."""
    s = re.sub(r'\s', '', s) # "whitespace is ignored"
    length = 0
    i = 0
    while i < len(s):
        m = matcher(s, i)
        if m:
            C, R = map(int, m.groups())
            i = m.end(0)                              # Advance to end of '(CxR)'
            length += R * decompress_length(s[i:i+C]) # Decompress C chars and add to length
            i += C                                    # Advance past the C characters 
        else:
            length += 1                               # Add 1 regular character to length
            i += 1                                    # Advance past it
    return length

print(decompress_length(Input(9).read()))

74532
11558231665


In [20]:
## Advent Day 10

def bots(instructions, goal={17, 61}):
    "Follow the data flow instructions, and if a bot gets the goal, print it."
    def give(giver, chip, recip):
        "Pass the chip from giver to recipient."
        has[giver].discard(chip)
        has[recip].add(chip)
        chips = has[recip]
        if chips == goal:
            print(recip, 'has', goal)
        if len(chips) == 2:
            give(recip, min(chips), gives[recip][0])
            give(recip, max(chips), gives[recip][1])
            
    has   = defaultdict(set)       # who has what
    gives = {giver: (dest1, dest2) # who will give what
             for (giver, dest1, dest2) 
             in re.findall(r'(bot \d+) gives low to (\w+ \d+) and high to (\w+ \d+)', instructions)}
    for (chip, recip) in re.findall(r'value (\d+) goes to (\w+ \d+)', instructions):
        give('input bin', int(chip), recip)
    return has

has = bots(Input(10).read())


def out(i): return has['output ' + str(i)].pop()

print(out(0) * out(1) * out(2))

bot 157 has {17, 61}
1085


In [23]:
## Advent Day 11

State = namedtuple('State', 'elevator, floors')

def fs(*items): return frozenset(items)

legal_floors = {0, 1, 2, 3}

def combos(things):
    "All subsets of 1 or 2 things."
    for s in chain(combinations(things, 1), combinations(things, 2)):
        yield fs(*s)

def moves(state):
    "All legal states that can be reached in one move from this state"
    L, floors = state
    for L2 in {L + 1, L - 1} & legal_floors:
        for stuff in combos(floors[L]):
            newfloors = tuple((s | stuff if i == L2 else 
                               s - stuff if i == state.elevator else 
                               s)
                              for (i, s) in enumerate(state.floors))
            if legal_floor(newfloors[L]) and legal_floor(newfloors[L2]):
                yield State(L2, newfloors)

def legal_floor(floor):
    "Floor is legal if no RTG, or every chip has its corresponding RTG."
    rtgs  = any(r.endswith('G') for r in floor)
    chips = [c for c in floor if c.endswith('M')]
    return not rtgs or all(generator_for(c) in floor for c in chips)

def generator_for(chip): return chip[0] + 'G'

def h_to_top(state):
    "An estimate of the number of moves needed to move everything to top."
    total = sum(len(floor) * i for (i, floor) in enumerate(reversed(state.floors)))
    return math.ceil(total / 2) # Can move two items in one move.

part1 = State(0, (fs('TG', 'TM', 'PG', 'SG'), fs('PM', 'SM'), fs('pM', 'pG', 'RM', 'RG'), Ø))

%time path = astar_search(part1, h_to_top, moves)
print(len(path) - 1)

part2 = State(0, (fs('TG', 'TM', 'PG', 'SG', 'EG', 'EM', 'DG', 'DM'), 
                  fs('PM', 'SM'), fs('pM', 'pG', 'RM', 'RG'), Ø))

%time path = astar_search(part2, h_to_top, moves)
print(len(path) - 1)

Wall time: 13.2 s
31
Wall time: 15min 6s
55


In [25]:
## Advent Day 12

def interpret(code, regs):
    "Execute instructions until pc goes off the end."
    def val(x): return (regs[x] if x in regs else x)
    pc = 0
    while pc < len(code):
        inst = code[pc]
        op, x, y = inst[0], inst[1], inst[-1]
        pc += 1
        if   op == 'cpy': regs[y] = val(x)
        elif op == 'inc': regs[x] += 1
        elif op == 'dec': regs[x] -= 1
        elif op == 'jnz' and val(x): pc += y - 1
    return regs

def parse(line): 
    "Split line into words, and convert to int where appropriate."
    return tuple((x if x.isalpha() else int(x)) 
                 for x in line.split())

code = [parse(line) for line in Input(12)]

print(interpret(code, dict(a=0, b=0, c=0, d=0)))
print(interpret(code, dict(a=0, b=0, c=1, d=0)))

{'a': 318003, 'b': 196418, 'c': 0, 'd': 0}
{'a': 9227657, 'b': 5702887, 'c': 0, 'd': 0}


In [27]:
## Advent Day 13

favorite = 1350
goal = (31, 39)

def is_open(location):
    "Is this an open location?"
    x, y = location
    num = x*x + 3*x + 2*x*y + y + y*y + favorite
    return x >= 0 and y >= 0 and bin(num).count('1') % 2 == 0

def open_neighbors(location): return filter(is_open, neighbors4(location))

path = astar_search((1, 1), lambda p: cityblock_distance(p, goal), open_neighbors)
print(len(path) - 1)

def count_locations_within(start, N, neighbors):
    "Find how many locations are within N steps from start."
    frontier = deque([start]) # A queue of states
    distance = {start: 0}     # distance to start; also tracks all states seen
    while frontier:
        s = frontier.popleft()
        if distance[s] < N:
            for s2 in neighbors(s):
                if s2 not in distance:
                    frontier.append(s2)
                    distance[s2] = distance[s] + 1
    return len(distance)
                
print(count_locations_within((1, 1), 50, open_neighbors))

92
124


In [28]:
## Advent Day 14

salt = 'ahsbgdzn'

@lru_cache(1001)
def hashval(i): return hashlib.md5(bytes(salt + str(i), 'utf-8')).hexdigest()

def is_key(i):
    "A key has a triple like '777', and then '77777' in one of the next thousand hashval(i)."
    three = re.search(r'(.)\1\1', hashval(i))
    if three:
        five = three.group(1) * 5
        return any(five in hashval(i+delta) for delta in range(1, 1001))
    
def nth_key(N): return nth(filter(is_key, range(BIG)), N)

print(nth_key(63))

@lru_cache(1001)
def hashval(i, stretch=2016): 
    h = hashlib.md5(bytes(salt + str(i), 'utf-8')).hexdigest()
    for i in range(stretch):
        h = hashlib.md5(bytes(h, 'utf-8')).hexdigest()
    return h

%time print(nth_key(63))

23890
22696
Wall time: 43.5 s


In [31]:
## Advent Day 15

def parse(inputs): 
    "Parse an input string into (disc#, positions, pos) triples."
    return [tuple(map(int, triple))
            for triple in re.findall(r'#(\d+).* (\d+) positions.* (\d+)[.]', inputs)]
    
discs = parse(Input(15).read())

def falls(t, discs):
    "If we drop the capsule at time t, does it fall through all slots?"
    return all((pos + t + d) % positions == 0 
               for (d, positions, pos) in discs)

print(first(t for t in range(BIG) if falls(t, discs)))

discs.append((7, 11, 0))

print(first(t for t in range(12, BIG, 19) if falls(t, discs)))

400589
3045959


In [34]:
## Advent Day 16

def expand(a, N):
    "Expand seed `a` until it has length N."
    while len(a) < N:
        b = flip(a[::-1])
        a = a + '0' + b
    return a[:N]

def flip(text, table=str.maketrans('10', '01')): return text.translate(table)

def checksum(a):
    "Compute the checksum of `a` by comparing pairs until len is odd."
    while len(a) % 2 == 0:
        a = cat(('1' if a[i] == a[i+1] else '0') 
                for i in range(0, len(a), 2))
    return a
    
seed = '10111011111001111'

print(checksum(expand(seed, 272)))

%time print(checksum(expand(seed, 35651584)))

11101010111100010
01001101001000101
Wall time: 7.14 s


In [36]:
## Advent Day 17

passcode = 'vkjiggvb'

openchars = 'bcdef'

grid = set((x, y) for x in range(4) for y in range(4))

start, goal = (0, 0), (3, 3)

def to_goal(state): 
    "City block distance between state's position and goal."
    pos, path = state
    return cityblock_distance(pos, goal)

directions = [(0, 'U', (0, -1)), (1, 'D', (0, 1)), (2, 'L', (-1, 0)), (3, 'R', (1, 0))]

def moves(state):
    "All states reachable from this state."
    (x, y), path = state
    hashx = hashlib.md5(bytes(passcode + path, 'utf-8')).hexdigest()
    for (i, p, (dx, dy)) in directions:
        pos2 = (x+dx, y+dy)
        if hashx[i] in openchars and pos2 in grid:
            yield (pos2, path+p)
    
print(astar_search((start, ''), to_goal, moves))

def longest_search(state, goal, moves):
    "Find the longest path to goal by depth-first search."
    longest = 0
    frontier = [state]
    while frontier:
        state = (pos, path) = frontier.pop()
        if pos == goal:
            longest = max(longest, len(path))
        else:
            frontier.extend(moves(state))
    return longest
            
print(longest_search((start, ''), goal, moves))

[((0, 0), ''), ((1, 0), 'R'), ((1, 1), 'RD'), ((2, 1), 'RDR'), ((3, 1), 'RDRR'), ((3, 0), 'RDRRU'), ((2, 0), 'RDRRUL'), ((2, 1), 'RDRRULD'), ((2, 2), 'RDRRULDD'), ((2, 3), 'RDRRULDDD'), ((3, 3), 'RDRRULDDDR')]
392


In [40]:
## Advent Day 18

safe, trap = '.', '^'  
initial = Input(18).read()

def rows(n, row=initial):
    "The first n rows of tiles (given the initial row)."
    result = [row]
    for i in range(n-1):
        previous = safe + result[-1] + safe
        result.append(cat((trap if previous[i-1] != previous[i+1] else safe)
                          for i in range(1, len(previous) - 1)))
    return result

print(cat(rows(40)).count(safe))

%time print(cat(rows(400000)).count(safe))

2005
20008491
Wall time: 7.82 s


In [42]:
## Advent Day 19

def Elves(N=3004953): return range(1, N+1) 

def winner(elves): return (elves[0] if (len(elves) == 1) else winner(one_round(elves)))

def one_round(elves): return (elves[0::2] if (len(elves) % 2 == 0) else elves[2::2])

assert winner(Elves(5)) == 3

print(winner(Elves()))

def Elves(N=3004953): return list(range(1, N+1))

def one_round(elves):
    "The first third of elves eliminate ones across the circle from them; who is left?"
    N = len(elves)
    eliminated = 0
    for i in range(int(math.ceil(N / 3))):
        across = i + eliminated + (N // 2) 
        elves[across] = None
        N -= 1
        eliminated += 1
    return list(filter(None, elves[i+1:] + elves[:i+1]))

assert winner(Elves(5)) == 2

assert one_round(Elves(5)) == [4, 1, 2]

%time print(winner(Elves()))

1815603
1410630
Wall time: 1.31 s


In [44]:
## Advent Day 20

pairs = sorted(map(parse_ints, Input(20)))

def unblocked(pairs):
    "Find the lowest unblocked integer, given the sorted pairs of blocked numbers."
    i = 0
    for (low, high) in pairs:
        yield from range(i, low)
        i = max(i, high + 1)
            
print(first(unblocked(pairs)))

print(len(list(unblocked(pairs))))

22887907
109


In [46]:
## Advent Day 21

def scramble(password, instructions=list(Input(21)), verbose=False):
    "Scramble the password according to the instructions."
    pw = list(password) 
    def rot(N):     pw[:]        = pw[-N:] + pw[:-N]
    def swap(A, B): pw[A], pw[B] = pw[B], pw[A]  
    for line in instructions:
        words = line.split()
        A, B, = parse_ints(line + ' 0 0')[:2]
        cmd = line.startswith
        if   cmd('swap position'): swap(A, B)
        elif cmd('swap letter'):   swap(pw.index(words[2]), pw.index(words[5]))
        elif cmd('rotate right'):  rot(A)
        elif cmd('rotate left'):   rot(-A)
        elif cmd('reverse'):       pw[A:B+1] = pw[A:B+1][::-1]
        elif cmd('move'):          pw[A:A+1], pw[B:B] = [], pw[A:A+1]
        elif cmd('rotate based'):
            i = pw.index(words[6])
            rot((i + 1 + (i >= 4)) % len(pw))
        if verbose: 
            print(line + ': ' + cat(pw))
    return cat(pw)

print(scramble('abcdefgh'))

print({cat(p) for p in permutations('fbgdceah') 
 if scramble(p) == 'fbgdceah'})

dgfaehcb
{'fdhgacbe'}


In [48]:
## Advent Day 22

Node = namedtuple('Node', 'x, y, size, used, avail, pct')

nodes = [Node(*parse_ints(line)) for line in Input(22) if line.startswith('/dev')]

def viable(A, B): return A != B and 0 < A.used <= B.avail

print(sum(viable(A, B) for A in nodes for B in nodes))

empty = first(node for node in nodes if node.used == 0)
maxx  = max(node.x for node in nodes)

grid = {(node.x, node.y): node 
        for node in nodes}

State = namedtuple('State', 'datapos, emptypos')

def distance(state): return cityblock_distance(state.datapos)

def moves(state):
    "Try moving any neighbor we can into the empty position."
    for pos in neighbors4(state.emptypos):
        if pos in grid:
            # Try to move contents of `node` at pos into `empty` at emptypos
            node, empty = grid[pos], grid[state.emptypos]
            if node.used <= empty.size:
                newdatapos = (state.emptypos if pos == state.datapos else state.datapos)
                yield State(newdatapos, pos)
                        
path = astar_search(State((maxx, 0), (empty.x, empty.y)), distance, moves)
print(len(path) - 1)

888
236


In [5]:
## Advent Day 23

def interpret(code, regs):
    "Execute instructions until pc goes off the end."
    def val(x): return (regs[x] if x in regs else x)
    pc = 0
    while 0 <= pc < len(code):
        inst = code[pc]
        op, x, y = inst[0], inst[1], inst[-1]
        pc += 1
        if   op == 'cpy' and y in regs: regs[y] = val(x)
        elif op == 'inc':               regs[x] += 1                  
        elif op == 'dec':               regs[x] -= 1                  
        elif op == 'jnz' and val(x):    pc += val(y) - 1   
        elif op == 'tgl':               toggle(code, pc - 1 + val(x))
    return regs

def toggle(code, i):
    "Toggle the instruction at location i."
    if 0 <= i < len(code): 
        inst = code[i]
        inst[0] = ('dec' if inst[0] == 'inc' else 
                   'inc' if len(inst) == 2 else
                   'cpy' if inst[0] == 'jnz' else 
                   'jnz')

def parse(line): 
    "Split line into words, and convert to int where appropriate."
    return [(x if x.isalpha() else int(x)) 
            for x in line.split()]

text = Input(23).read().strip()

code = [parse(line) for line in text.splitlines()]

regs = dict(a=7, b=0, c=0, d=0)

print(interpret(code, regs))

code = [parse(line) for line in text.splitlines()]

regs = dict(a=12, b=0, c=0, d=0)

%time print(interpret(code, regs))

{'a': 12573, 'b': 1, 'c': 0, 'd': 0}
Wall time: 26min 3s


{'a': 479009133, 'b': 1, 'c': 0, 'd': 0}

In [7]:
maze = tuple(Input(24))
zero = first((x, y) for y, row in enumerate(maze) for x, c in enumerate(row) if c == '0')
def h(state):
    "Heuristic: the number of digits not yet visited."
    _, visited = state
    return 8 - len(visited) # Note: 8 == len('01234567')
    
def moves(state):
    "Move to any neighboring square that is not a wall. Track the digits visited."
    pos, visited = state
    for x1, y1 in neighbors4(pos):
        c = maze[y1][x1]
        if c != '#':
            visited1 = (visited if c in visited or c == '.' else cat(sorted(visited + c)))
            yield (x1, y1), visited1

path = astar_search((zero, '0'), h, moves)
print(len(path) - 1)

def h2(state):
    "Heuristic: the number of digits not yet visited, plus the distance back to start."
    pos, visited = state
    return 8 - len(visited) + cityblock_distance(pos, zero)

path2 = astar_search((zero, '0'), h2, moves)
print(len(path2) - 1)

456
704


In [11]:
def interpret(code, regs, steps=BIG):
    "Execute instructions until pc goes off the end, or until we execute the given number of steps."
    def val(x): return (regs[x] if x in regs else x)
    pc = 0
    for _ in range(steps):
        if not (0 <= pc < len(code)):
            return
        inst = code[pc]
        op, x, y = inst[0], inst[1], inst[-1]
        pc += 1
        if   op == 'cpy' and y in regs: regs[y] = val(x)
        elif op == 'inc':               regs[x] += 1                  
        elif op == 'dec':               regs[x] -= 1                  
        elif op == 'jnz' and val(x):    pc += val(y) - 1   
        elif op == 'tgl':               toggle(code, pc - 1 + val(x))
        elif op == 'out':               yield val(x)
            
text = Input(25).read().strip()

code = [parse(line) for line in text.splitlines()]

def repeats(a, code, steps=10**6, minsignals=100):
    "Does this value for register a cause code to repeat `out` signals of 0, 1, 0, 1, ...?"
    signals = interpret(code, dict(a=a, b=0, c=0, d=0), steps)
    expecteds = cycle((0, 1))
    for (i, (signal, expected)) in enumerate(zip(signals, expecteds)):
        if signal != expected:
            return False
    # We'll say "yes" if the code outputs at least a minimum number of 0, 1, ... signals, and nothing else.
    return i >= minsignals
    
first(a for a in range(1, BIG) if repeats(a, code))

192